<a href="https://colab.research.google.com/github/zabihin/Project_Kayak/blob/main/Bloc_1_Plan_your_trip_with_Kayak_Zahra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

⚠️ This project is mandatory for certification bloc #1.

![Kayak](https://seekvectorlogo.com/wp-content/uploads/2018/01/kayak-vector-logo.png)

# Plan your trip with Kayak 

## Company's description 📇

<a href="https://www.kayak.com" target="_blank">Kayak</a> is a travel search engine that helps user plan their next trip at the best price.

The company was founded in 2004 by Steve Hafner & Paul M. English. After a few rounds of fundraising, Kayak was acquired by <a href="https://www.bookingholdings.com/" target="_blank">Booking Holdings</a> which now holds: 

* <a href="https://booking.com/" target="_blank">Booking.com</a>
* <a href="https://kayak.com/" target="_blank">Kayak</a>
* <a href="https://www.priceline.com/" target="_blank">Priceline</a>
* <a href="https://www.agoda.com/" target="_blank">Agoda</a>
* <a href="https://Rentalcars.com/" target="_blank">RentalCars</a>
* <a href="https://www.opentable.com/" target="_blank">OpenTable</a>

With over \$300 million revenue a year, Kayak operates in almost all countries and all languages to help their users book travels accros the globe. 

## Project 🚧

The marketing team needs help on a new project. After doing some user research, the team discovered that **70% of their users who are planning a trip would like to have more information about the destination they are going to**. 

In addition, user research shows that **people tend to be defiant about the information they are reading if they don't know the brand** which produced the content. 

Therefore, Kayak Marketing Team would like to create an application that will recommend where people should plan their next holidays. The application should be based on real data about:

* Weather 
* Hotels in the area 

The application should then be able to recommend the best destinations and hotels based on the above variables at any given time. 

## Goals 🎯

As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to: 

* Scrape data from destinations 
* Get weather data from each destination 
* Get hotels' info about each destination
* Store all the information above in a data lake
* Extract, transform and load cleaned data from your datalake to a data warehouse

## Scope of this project 🖼️

Marketing team wants to focus first on the best cities to travel to in France. According <a href="https://one-week-in.com/35-cities-to-visit-in-france/" target="_blank">One Week In.com</a> here are the top-35 cities to visit in France: 

```python 
["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]
```

Your team should focus **only on the above cities for your project**. 


## Helpers 🦮

To help you achieve this project, here are a few tips that should help you

### Get weather data with an API 

*   Use https://nominatim.org/ to get the gps coordinates of all the cities (no subscription required) Documentation : https://nominatim.org/release-docs/develop/api/Search/

*   Use https://openweathermap.org/appid (you have to subscribe to get a free apikey) and https://openweathermap.org/api/one-call-api to get some information about the weather for the 35 cities and put it in a DataFrame

*   Determine the list of cities where the weather will be the nicest within the next 7 days For example, you can use the values of daily.pop and daily.rain to compute the expected volume of rain within the next 7 days... But it's only an example, actually you can have different opinions on a what a nice weather would be like 😎 Maybe the most important criterion for you is the temperature or humidity, so feel free to change the rules !

*   Save all the results in a `.csv` file, you will use it later 😉 You can save all the informations that seem important to you ! Don't forget to save the name of the cities, and also to create a column containing a unique identifier (id) of each city (this is important for what's next in the project)

*   Use plotly to display the best destinations on a map

### Scrape Booking.com 

Since BookingHoldings doesn't have aggregated databases, it will be much faster to scrape data directly from booking.com 

You can scrap as many information asyou want, but we suggest that you get at least:

*   hotel name,
*   Url to its booking.com page,
*   Its coordinates: latitude and longitude
*   Score given by the website users
*   Text description of the hotel


### Create your data lake using S3 

Once you managed to build your dataset, you should store into S3 as a csv file. 

### ETL 

Once you uploaded your data onto S3, it will be better for the next data analysis team to extract clean data directly from a Data Warehouse. Therefore, create a SQL Database using AWS RDS, extract your data from S3 and store it in your newly created DB. 

## Deliverable 📬

To complete this project, your team should deliver:

* A `.csv` file in an S3 bucket containing enriched information about weather and hotels for each french city

* A SQL Database where we should be able to get the same cleaned data from S3 

* Two maps where you should have a Top-5 destinations and a Top-20 hotels in the area. You can use plotly or any other library to do so. It should look something like this: 

![Map](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/Kayak_best_destination_project.png)

In [ ]:
!pip install Scrapy
!pip install Boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 127 kB 19.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.13
    Uninstalling urllib3-1.26.13:
      Successfully uninstalled urllib3-1.26.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###Import libraries

In [ ]:
from datetime import datetime, date, timedelta
import calendar
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

from bs4 import BeautifulSoup

import requests
import pandas as pd
import numpy as np
from statistics import mode
import plotly.express as px
import plotly.graph_objects as go



I Suppose a couple decided to go on a trip to in France this weekend. They want to choose one of the 35 cities in France based on the weather. I have assumed that if the average temperature of 35 cities is higher than 22 (probably summer) and the preference is with the coolest destination, but if the temperature is low, the preference is with the hottest city. Among 35 cities, 5 cities are selected according to the weather. 

find the date of next saturday and sunday for finding the weather and find hotels






In [ ]:
today = date.today() 
sat = today + timedelta((calendar.SATURDAY-today.weekday()) % 7 )
sun= today + timedelta((calendar.SUNDAY-today.weekday()) % 7 )



this is the list of best cities to travel in france. 

In [ ]:
#city list
best_cities=["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg","Chateau du Haut Koenigsbourg",
"Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon","Bormes les Mimosas", "Cassis",
"Marseille","Aix en Provence","Avignon","Uzes","Nimes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne",
"Ariege","Toulouse","Montauban","Biarritz","Bayonne","La Rochelle"]

we should find the location of each city (we need them for finding how weather will be this weekend)

In [ ]:
url = "https://nominatim.openstreetmap.org/search?"

def find_gps(address, format='json'):
   
    params = {"q":address, "format": format}
    response = requests.get(url, params=params)
    return response.json()
columns = ['id', 'name', 'latitude', 'longitude']
data = []

for count, city in enumerate(best_cities):
    response = find_gps(address = city)
    row =[count, city, response[0]['lat'], response[0]['lon']]
    data.append(row)
    
df_cities = pd.DataFrame(data=data, columns=columns)

In [ ]:
df_cities.head()


,id,name,latitude,longitude
0,0,Mont Saint Michel,48.6359541,-1.511459954959514
1,1,St Malo,48.649518,-2.0260409
2,2,Bayeux,49.2764624,-0.7024738
3,3,Le Havre,49.4938975,0.1079732
4,4,Rouen,49.4404591,1.0939658


find the temprature of this saturday and sunday 

my key for the website https://api.openweathermap.org

In [ ]:
key='2941f971df0a6e6e0569606b236025bf'

In [ ]:
if sat.weekday()-today.weekday()<0:
  next_weekend=sat.weekday()-today.weekday()+7
else:
  next_weekend=sat.weekday()-today.weekday()


In [ ]:
df_cities['Temperature']=None
df_cities['Temperature_max']=None
df_cities['Temperature_min']=None


df_cities['Humidity']=None


for ind in df_cities.id:

    lat = df_cities.latitude[ind]
    lon = df_cities.longitude[ind]
    r =  requests.get(f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&units=metric&appid={key}").json()
    weather = r['daily'][next_weekend:next_weekend+2]
    df_cities.iat[ind, df_cities.columns.get_loc('Temperature')] =np.mean([i['feels_like']['day'] for i in weather])
    df_cities.iat[ind, df_cities.columns.get_loc('Temperature_max')] =np.mean([i['temp']['max'] for i in weather])
    df_cities.iat[ind, df_cities.columns.get_loc('Temperature_min')] =np.mean([i['temp']['min'] for i in weather])
    df_cities.iat[ind, df_cities.columns.get_loc('Humidity')] =np.mean( [i['humidity'] for i in weather])



In [ ]:
df_cities.head()

,id,name,latitude,longitude,Temperature,Temperature_max,Temperature_min,Humidity
0,0,Mont Saint Michel,48.6359541,-1.511459954959514,1.695,4.95,-1.065,69.5
1,1,St Malo,48.649518,-2.0260409,1.525,5.96,1.905,71.5
2,2,Bayeux,49.2764624,-0.7024738,2.14,4.55,0.325,74.5
3,3,Le Havre,49.4938975,0.1079732,1.25,4.515,1.745,69.0
4,4,Rouen,49.4404591,1.0939658,1.015,3.07,-0.78,68.5


find the avrerage of tempreture(feel-like,max temp, min temp) in this week-end for list of cities
if the mean of tempreture is more than 22, I assume that the best destination are the ones which have minimum of max temrature in that day 
and 
if the mean of tempreture is less than 22, I assume that the best destination are the ones which have maximum of min temrature in that day 

In [ ]:
Tem_avr=df_cities['Temperature'].mean()

In [ ]:
df_cities['rank_weather']=None


In [ ]:
if (Tem_avr-22)>0:
  df_cities.loc[:, 'rank_weather'] = df_cities['Temperature_max'].rank(method='min').astype(int)
  df_cities = df_cities.sort_values(by=['Temperature_max'], ascending=[True]).reset_index(drop=True)
else:
  df_cities.loc[:, 'rank_weather'] = df_cities['Temperature_min'].rank(method='max').astype(int)
  df_cities = df_cities.sort_values(by=['Temperature_min'], ascending=[True]).reset_index(drop=True)



In [ ]:
df_cities.head()

,id,name,latitude,longitude,Temperature,Temperature_max,Temperature_min,Humidity,rank_weather
0,29,Ariege,42.9455368,1.4065544156065486,6.06,6.06,-2.075,58.0,1
1,9,Chateau du Haut Koenigsbourg,48.249489800000006,7.34429620253195,1.03,1.77,-1.62,81.0,2
2,0,Mont Saint Michel,48.6359541,-1.511459954959514,1.695,4.95,-1.065,69.5,3
3,4,Rouen,49.4404591,1.0939658,1.015,3.07,-0.78,68.5,4
4,6,Amiens,49.8941708,2.2956951,-1.305,2.105,-0.415,74.5,5


Top 5 cities based on the weather

In [ ]:
top5_city=df_cities[0:5]

In [ ]:
top5_city

,id,name,latitude,longitude,Temperature,Temperature_max,Temperature_min,Humidity,rank_weather
0,29,Ariege,42.9455368,1.4065544156065486,6.06,6.06,-2.075,58.0,1
1,9,Chateau du Haut Koenigsbourg,48.249489800000006,7.34429620253195,1.03,1.77,-1.62,81.0,2
2,0,Mont Saint Michel,48.6359541,-1.511459954959514,1.695,4.95,-1.065,69.5,3
3,4,Rouen,49.4404591,1.0939658,1.015,3.07,-0.78,68.5,4
4,6,Amiens,49.8941708,2.2956951,-1.305,2.105,-0.415,74.5,5


In [ ]:
best_5city=top5_city.name.to_list()

the date of travel for scraping booking.com

*   2 person with no kid
*   date next weekend just for one night
*   order of hotel : based of review_score_and_price






In [ ]:
date_in=today + timedelta(days=next_weekend)
date_out=today + timedelta(days=next_weekend+1)

In [ ]:
year_in=str(date_in.year)
month_in=str(date_in.month)
day_in=str(date_in.day)
month_out=str(date_out.month)
day_out=str(date_out.day)

In [ ]:
class BookSpider(scrapy.Spider):
    name = "hotels"
    # Starting URL
    start_urls = ['https://www.booking.com/index.fr.html']
    
    
    # Parse function for login
    def parse(self, response):
      for city in best_5city:
        # FormRequest used to login
        yield scrapy.FormRequest.from_response(
            response,
            formdata={'ss': city,"checkin_year":year_in,"checkin_month":month_in,"checkin_monthday":day_in,
            "checkout_year":year_in,"checkout_month":month_out,"checkout_monthday":day_out,"selected_currency":"EUR",
            "group_adults":"2","order":"review_score_and_price"},
            callback=self.after_search
        )

    # Callback used after login
    def after_search(self, response):
      for item in response.css("div.main-spacing"):
        yield {
                  'city': item.css("div.efdb2b543b h1.e1f827110f.d3a14d00da::text").getall()[0].split("\xa0")[0],
                  'hotel': item.css("div.fcab3ed991.a23c043802::text").getall(),
                  'url':item.css("h3.a4225678b2 a.e13098a59f::attr(href) ").getall(),
                  'price': item.css("span.bd73d13072.fcab3ed991::text").getall(),
                  'rating':item.css("div.b5cd09854e.d10a6220b4::text").getall(),
                  }




In [ ]:
filename = "book.json"

if filename in os.listdir():
        os.remove(filename)

process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        filename: {"format": "json"},
    },
    "AUTOTHROTTLE_ENABLED": True
})

process.crawl(BookSpider)
process.start()


INFO:scrapy.utils.log:Scrapy 2.7.1 started (bot: scrapybot)
2022-11-29 12:29:07 [scrapy.utils.log] INFO: Scrapy 2.7.1 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.0, Twisted 22.10.0, Python 3.7.15 (default, Oct 12 2022, 19:14:55) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.7 1 Nov 2022), cryptography 38.0.4, Platform Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
2022-11-29 12:29:07 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.0, Twisted 22.10.0, Python 3.7.15 (default, Oct 12 2022, 19:14:55) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.7 1 Nov 2022), cryptography 38.0.4, Platform Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
INFO:scrapy.crawler:Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) '
               'Gecko/20100101 Firefox/92.0'}
2022-11-29 12

In [ ]:
df = pd.read_json('book.json')


list of hotels, their webadress, price and rating for 5 cites

In [ ]:
df

,city,hotel,url,price,rating
0,Ariège,"[LE YESLOWBUS en fête, LOFT24, VILLA LES GLYCI...",[https://www.booking.com/hotel/fr/le-yeslowbus...,"[€ 91, € 210, € 307, € 476, € 82, € 89, € 97, ...","[10, 10, 9,5, 9,5, 9,5, 9,5, 9,5, 9,5, 9,6, 9,..."
1,Château du Haut-Kœnigsbourg,"[Le Colombier, AOG PRESTIGE Privat SPA Jacuzzi...",[https://www.booking.com/hotel/fr/colombiercol...,"[€ 1 000, € 1 013]","[8,4, 8,4]"
2,Le Mont-Saint-Michel,"[Mercure Mont Saint Michel, Le Relais Saint Mi...",[https://www.booking.com/hotel/fr/mont-saint-m...,"[€ 130, € 158, € 175, € 190, € 579, € 88, € 92...","[8,2, 7,8, 7,9, 7,6, 10, 9,5, 9,6, 9,5, 9,6, 9..."
3,Rouen,"[Ragnar, Villa jacuzzi privatif, 3DALIPHARD, L...",[https://www.booking.com/hotel/fr/ragnar.fr.ht...,"[€ 97, € 165, € 126, € 236, € 82, € 93, € 118,...","[10, 10, 9,5, 9,7, 9,2, 9,1, 9,1, 9,0, 9,3, 9,..."
4,Amiens,[Le Cadre d'Or - Appartement Confort - Paisibl...,[https://www.booking.com/hotel/fr/le-cadre-d-o...,"[€ 74, € 81, € 753, € 86, € 509, € 103, € 81, ...","[10, 10, 9,1, 9,4, 8,5, 8,0, 8,2, 8,2, 7,6, 7,..."


adding the other infomation of gps and weather to this dataframe


In [ ]:
df["latitude"]=top5_city["latitude"]
df["longitude"]=top5_city["longitude"]
df["Temperature"]=top5_city["Temperature"]
df["Temperature_max"]=top5_city["Temperature_max"]
df["Temperature_min"]=top5_city["Temperature_min"]
df["Humidity"]=top5_city["Humidity"]

df

,city,hotel,url,price,rating,latitude,longitude,Temperature,Temperature_max,Temperature_min,Humidity
0,Ariège,"[LE YESLOWBUS en fête, LOFT24, VILLA LES GLYCI...",[https://www.booking.com/hotel/fr/le-yeslowbus...,"[€ 91, € 210, € 307, € 476, € 82, € 89, € 97, ...","[10, 10, 9,5, 9,5, 9,5, 9,5, 9,5, 9,5, 9,6, 9,...",42.9455368,1.4065544156065486,6.06,6.06,-2.075,58.0
1,Château du Haut-Kœnigsbourg,"[Le Colombier, AOG PRESTIGE Privat SPA Jacuzzi...",[https://www.booking.com/hotel/fr/colombiercol...,"[€ 1 000, € 1 013]","[8,4, 8,4]",48.249489800000006,7.34429620253195,1.03,1.77,-1.62,81.0
2,Le Mont-Saint-Michel,"[Mercure Mont Saint Michel, Le Relais Saint Mi...",[https://www.booking.com/hotel/fr/mont-saint-m...,"[€ 130, € 158, € 175, € 190, € 579, € 88, € 92...","[8,2, 7,8, 7,9, 7,6, 10, 9,5, 9,6, 9,5, 9,6, 9...",48.6359541,-1.511459954959514,1.695,4.95,-1.065,69.5
3,Rouen,"[Ragnar, Villa jacuzzi privatif, 3DALIPHARD, L...",[https://www.booking.com/hotel/fr/ragnar.fr.ht...,"[€ 97, € 165, € 126, € 236, € 82, € 93, € 118,...","[10, 10, 9,5, 9,7, 9,2, 9,1, 9,1, 9,0, 9,3, 9,...",49.4404591,1.0939658,1.015,3.07,-0.78,68.5
4,Amiens,[Le Cadre d'Or - Appartement Confort - Paisibl...,[https://www.booking.com/hotel/fr/le-cadre-d-o...,"[€ 74, € 81, € 753, € 86, € 509, € 103, € 81, ...","[10, 10, 9,1, 9,4, 8,5, 8,0, 8,2, 8,2, 7,6, 7,...",49.8941708,2.2956951,-1.305,2.105,-0.415,74.5


In [ ]:
best1={"name":df.city[0],"hotel":df.hotel[0][0:20],"url":df.url[0][0:20],"price":df.price[0][0:20],"rating":df.rating[0][0:20],
       "Temperature":df.Temperature[0],"Temperature_max":df.Temperature_max[0],"Temperature_min":df.Temperature_min[0],"Humidity":df.Humidity[0]
       }
df_1=pd.DataFrame(best1)
df_1


,name,hotel,url,price,rating,Temperature,Temperature_max,Temperature_min,Humidity
0,Ariège,LE YESLOWBUS en fête,https://www.booking.com/hotel/fr/le-yeslowbus-...,€ 91,10,6.06,6.06,-2.075,58.0
1,Ariège,LOFT24,https://www.booking.com/hotel/fr/loft24.fr.htm...,€ 210,10,6.06,6.06,-2.075,58.0
2,Ariège,VILLA LES GLYCINES PYRENEES ARIEGOISES,https://www.booking.com/hotel/fr/villa-les-gly...,€ 307,"9,5",6.06,6.06,-2.075,58.0
3,Ariège,Maison de 3 chambres avec jardin amenage et wi...,https://www.booking.com/hotel/fr/house-with-3-...,€ 476,"9,5",6.06,6.06,-2.075,58.0
4,Ariège,Belloreade,https://www.booking.com/hotel/fr/belloreade.fr...,€ 82,"9,5",6.06,6.06,-2.075,58.0
5,Ariège,Chambres d'hôtes St Jacques Adults only,https://www.booking.com/hotel/fr/chambre-d-39-...,€ 89,"9,5",6.06,6.06,-2.075,58.0
6,Ariège,Maison d'hotes et Gîte près de Saint Girons - ...,https://www.booking.com/hotel/fr/chambre-d-hot...,€ 97,"9,5",6.06,6.06,-2.075,58.0
7,Ariège,Domaine Le Lanis Chambre d'hôtes avec piscine ...,https://www.booking.com/hotel/fr/domaine-du-la...,€ 128,"9,5",6.06,6.06,-2.075,58.0
8,Ariège,Eau Berges,https://www.booking.com/hotel/fr/eau-berges.fr...,€ 144,"9,6",6.06,6.06,-2.075,58.0
9,Ariège,gite le Clot,https://www.booking.com/hotel/fr/gite-le-clot....,€ 139,"9,5",6.06,6.06,-2.075,58.0


In [ ]:
best2={"name":df.city[1],"hotel":df.hotel[1][0:20],"url":df.url[1][0:20],"price":df.price[1][0:20],"rating":df.rating[1][0:20],
       "Temperature":df.Temperature[1],"Temperature_max":df.Temperature_max[1],"Temperature_min":df.Temperature_min[1],"Humidity":df.Humidity[1]}
df_2=pd.DataFrame(best2)

In [ ]:
best3={"name":df.city[2],"hotel":df.hotel[2][0:20],"url":df.url[2][0:20],"price":df.price[2][0:20],"rating":df.rating[2][0:20],
       "Temperature":df.Temperature[2],"Temperature_max":df.Temperature_max[2],"Temperature_min":df.Temperature_min[2],"Humidity":df.Humidity[2]}
df_3=pd.DataFrame(best3)

In [ ]:
best4={"name":df.city[3],"hotel":df.hotel[3][0:20],"url":df.url[3][0:20],"price":df.price[3][0:20],"rating":df.rating[3][0:20],
       "Temperature":df.Temperature[3],"Temperature_max":df.Temperature_max[3],"Temperature_min":df.Temperature_min[3],"Humidity":df.Humidity[3]}
df_4=pd.DataFrame(best4)

In [ ]:
best5={"name":df.city[4],"hotel":df.hotel[4][0:20],"url":df.url[4][0:20],"price":df.price[4][0:20],"rating":df.rating[4][0:20],
       "Temperature":df.Temperature[4],"Temperature_max":df.Temperature_max[4],"Temperature_min":df.Temperature_min[4],"Humidity":df.Humidity[4]}
df_5=pd.DataFrame(best5)

In [ ]:
df_topp5=df_1.append((df_2,df_3,df_4,df_5), ignore_index = True)


In [ ]:
df_topp5

,name,hotel,url,price,rating,Temperature,Temperature_max,Temperature_min,Humidity
0,Ariège,LE YESLOWBUS en fête,https://www.booking.com/hotel/fr/le-yeslowbus-...,€ 91,10,6.060,6.060,-2.075,58.0
1,Ariège,LOFT24,https://www.booking.com/hotel/fr/loft24.fr.htm...,€ 210,10,6.060,6.060,-2.075,58.0
2,Ariège,VILLA LES GLYCINES PYRENEES ARIEGOISES,https://www.booking.com/hotel/fr/villa-les-gly...,€ 307,"9,5",6.060,6.060,-2.075,58.0
3,Ariège,Maison de 3 chambres avec jardin amenage et wi...,https://www.booking.com/hotel/fr/house-with-3-...,€ 476,"9,5",6.060,6.060,-2.075,58.0
4,Ariège,Belloreade,https://www.booking.com/hotel/fr/belloreade.fr...,€ 82,"9,5",6.060,6.060,-2.075,58.0
...,...,...,...,...,...,...,...,...,...
77,Amiens,Petit cocon idéal pour court ou long séjour,https://www.booking.com/hotel/fr/petit-cocon-i...,€ 59,"6,5",-1.305,2.105,-0.415,74.5
78,Amiens,Comfort Hotel Amiens Nord,https://www.booking.com/hotel/fr/kyriad-amiens...,€ 66,"6,5",-1.305,2.105,-0.415,74.5
79,Amiens,Appartement Cosy 5 Amiens,https://www.booking.com/hotel/fr/appartement-c...,€ 73,"6,6",-1.305,2.105,-0.415,74.5
80,Amiens,Ever green,https://www.booking.com/hotel/fr/ever-green.fr...,€ 79,"6,0",-1.305,2.105,-0.415,74.5


In [ ]:
df_topp5.rating=df_topp5.rating.apply(lambda x: x.replace(",","."))
df_topp5.price=df_topp5.price.apply(lambda x: x.split("€")[1])



In [ ]:
df_topp5['lon_hotel']=None
df_topp5['lat_hotel']=None
df_topp5['address_hotel']=None

In [ ]:
for ind in df_topp5.index:
  print(ind)
  url=df_topp5.url[ind]
  page = requests.get(url)
  soup = BeautifulSoup(page.content, "lxml")
  df_topp5.lat_hotel[ind]=soup.find( class_ = "jq_tooltip loc_block_link_underline_fix bui-link show_on_map_hp_link show_map_hp_link " ).get("data-atlas-latlng").split(',')[0]
  df_topp5.lon_hotel[ind]=soup.find( class_ = "jq_tooltip loc_block_link_underline_fix bui-link show_on_map_hp_link show_map_hp_link " ).get("data-atlas-latlng").split(',')[1]
  df_topp5.address_hotel[ind]=soup.find(class_ = " hp_address_subtitle js-hp_address_subtitle jq_tooltip ").getText().strip('\n')


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.booking.com:443


0


DEBUG:urllib3.connectionpool:https://www.booking.com:443 "GET /hotel/fr/le-yeslowbus-en-fete.fr.html?label=gen173nr-1FCAEoggI46AdIDVgEaMkBiAEBmAENuAEYyAEP2AEB6AEB-AECiAIBqAIEuAKT9pecBsACAdICJDdjOTRlNzE4LWIxNmUtNGM4OS1iZDM3LTMzMzM1YjlmOTA0MdgCBeACAQ&sid=5eea383651bcf9fa9be88807c19ecdbb&aid=304142&ucfs=1&arphpl=1&checkin=2022-12-03&checkout=2022-12-04&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=review_score_and_price&srpvid=d85657cbaf0802fe&srepoch=1669724952&all_sr_blocks=898265501_360475080_2_0_0&highlighted_blocks=898265501_360475080_2_0_0&matching_block_id=898265501_360475080_2_0_0&sr_pri_blocks=898265501_360475080_2_0_0__9070&from=searchresults HTTP/1.1" 301 None
DEBUG:urllib3.connectionpool:https://www.booking.com:443 "GET /hotel/fr/le-yeslowbus-en-fete.fr.html HTTP/1.1" 200 248973
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

1


DEBUG:urllib3.connectionpool:https://www.booking.com:443 "GET /hotel/fr/loft24.fr.html?label=gen173nr-1FCAEoggI46AdIDVgEaMkBiAEBmAENuAEYyAEP2AEB6AEB-AECiAIBqAIEuAKT9pecBsACAdICJDdjOTRlNzE4LWIxNmUtNGM4OS1iZDM3LTMzMzM1YjlmOTA0MdgCBeACAQ&sid=5eea383651bcf9fa9be88807c19ecdbb&aid=304142&ucfs=1&arphpl=1&checkin=2022-12-03&checkout=2022-12-04&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=2&hapos=2&sr_order=review_score_and_price&srpvid=d85657cbaf0802fe&srepoch=1669724952&all_sr_blocks=844947701_351628896_2_0_0&highlighted_blocks=844947701_351628896_2_0_0&matching_block_id=844947701_351628896_2_0_0&sr_pri_blocks=844947701_351628896_2_0_0__20950&from_sustainable_property_sr=1&from=searchresults HTTP/1.1" 301 None
DEBUG:urllib3.connectionpool:https://www.booking.com:443 "GET /hotel/fr/loft24.fr.html HTTP/1.1" 200 280332
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.booking.com:443


...

DEBUG:urllib3.connectionpool:https://www.booking.com:443 "GET /hotel/fr/appartement-cosy-5-amiens.fr.html?label=gen173nr-1FCAEoggI46AdIDVgEaMkBiAEBmAENuAEYyAEP2AEB6AEB-AECiAIBqAIEuAKT9pecBsACAdICJDdjOTRlNzE4LWIxNmUtNGM4OS1iZDM3LTMzMzM1YjlmOTA0MdgCBeACAQ&sid=5eea383651bcf9fa9be88807c19ecdbb&aid=304142&ucfs=1&arphpl=1&checkin=2022-12-03&checkout=2022-12-04&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=18&hapos=18&sr_order=review_score_and_price&srpvid=e53257cf9de20492&srepoch=1669724960&all_sr_blocks=746113701_330550365_2_0_0&highlighted_blocks=746113701_330550365_2_0_0&matching_block_id=746113701_330550365_2_0_0&sr_pri_blocks=746113701_330550365_2_0_0__7260&from=searchresults HTTP/1.1" 301 None
DEBUG:urllib3.connectionpool:https://www.booking.com:443 "GET /hotel/fr/appartement-cosy-5-amiens.fr.html HTTP/1.1" 200 272905
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.booking.com:443


80


DEBUG:urllib3.connectionpool:https://www.booking.com:443 "GET /hotel/fr/ever-green.fr.html?label=gen173nr-1FCAEoggI46AdIDVgEaMkBiAEBmAENuAEYyAEP2AEB6AEB-AECiAIBqAIEuAKT9pecBsACAdICJDdjOTRlNzE4LWIxNmUtNGM4OS1iZDM3LTMzMzM1YjlmOTA0MdgCBeACAQ&sid=5eea383651bcf9fa9be88807c19ecdbb&aid=304142&ucfs=1&arphpl=1&checkin=2022-12-03&checkout=2022-12-04&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=19&hapos=19&sr_order=review_score_and_price&srpvid=e53257cf9de20492&srepoch=1669724960&all_sr_blocks=855653301_353098440_2_0_0&highlighted_blocks=855653301_353098440_2_0_0&matching_block_id=855653301_353098440_2_0_0&sr_pri_blocks=855653301_353098440_2_0_0__7948&from=searchresults HTTP/1.1" 301 None
DEBUG:urllib3.connectionpool:https://www.booking.com:443 "GET /hotel/fr/ever-green.fr.html HTTP/1.1" 200 257030
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.booking.com:443


81


DEBUG:urllib3.connectionpool:https://www.booking.com:443 "GET /hotel/fr/premia-re-classe-amiens.fr.html?label=gen173nr-1FCAEoggI46AdIDVgEaMkBiAEBmAENuAEYyAEP2AEB6AEB-AECiAIBqAIEuAKT9pecBsACAdICJDdjOTRlNzE4LWIxNmUtNGM4OS1iZDM3LTMzMzM1YjlmOTA0MdgCBeACAQ&sid=5eea383651bcf9fa9be88807c19ecdbb&aid=304142&ucfs=1&arphpl=1&checkin=2022-12-03&checkout=2022-12-04&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=20&hapos=20&sr_order=review_score_and_price&srpvid=e53257cf9de20492&srepoch=1669724960&all_sr_blocks=116371301_250733849_2_2_0&highlighted_blocks=116371301_250733849_2_2_0&matching_block_id=116371301_250733849_2_2_0&sr_pri_blocks=116371301_250733849_2_2_0__5468&from=searchresults HTTP/1.1" 301 None
DEBUG:urllib3.connectionpool:https://www.booking.com:443 "GET /hotel/fr/premia-re-classe-amiens.fr.html HTTP/1.1" 200 275070


In [ ]:
df_topp5.head()

,name,hotel,url,price,rating,Temperature,Temperature_max,Temperature_min,Humidity,lon_hotel,lat_hotel,address_hotel
0,Ariège,LE YESLOWBUS en fête,https://www.booking.com/hotel/fr/le-yeslowbus-...,91,10,6.06,6.06,-2.075,58.0,1.37339700,43.15829880,"Saubole, 09130 Carla-Bayle, France"
1,Ariège,LOFT24,https://www.booking.com/hotel/fr/loft24.fr.htm...,210,10,6.06,6.06,-2.075,58.0,1.61830160,42.83375580,"24 Promenade du Ruisseau, 09400 Ussat, France"
2,Ariège,VILLA LES GLYCINES PYRENEES ARIEGOISES,https://www.booking.com/hotel/fr/villa-les-gly...,307,9.5,6.06,6.06,-2.075,58.0,1.76089850,42.76187640,"4 Rue Principale, 09250 Luzenac, France"
3,Ariège,Maison de 3 chambres avec jardin amenage et wi...,https://www.booking.com/hotel/fr/house-with-3-...,476,9.5,6.06,6.06,-2.075,58.0,1.89387700,42.87794800,"Le Rasteille Ariège,, 09300 Fougax-et-Barrineu..."
4,Ariège,Belloreade,https://www.booking.com/hotel/fr/belloreade.fr...,82,9.5,6.06,6.06,-2.075,58.0,1.92531400,43.01598500,"Chemin du Carme, 09600 Belloc, France"


In [ ]:
df_topp5.to_csv('df_topp5' , encoding='utf-8', index=False)


#Data storage

In [ ]:
import boto3


In [ ]:
 ##Using boto3 to create and connect a S3 bucket

session = boto3.Session(aws_access_key_id='**', aws_secret_access_key='********')
s3 = session.resource('s3')



DEBUG:botocore.hooks:Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
DEBUG:botocore.hooks:Changing event name from before-call.apigateway to before-call.api-gateway
DEBUG:botocore.hooks:Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
DEBUG:botocore.hooks:Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration
DEBUG:botocore.hooks:Changing event name from before-parameter-build.route53 to before-parameter-build.route-53
DEBUG:botocore.hooks:Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.Search
DEBUG:botocore.hooks:Changing event name from docs.*.autoscaling.CreateLaunchConfiguration.complete-section to docs.*.auto-scaling.CreateLaunchConfiguration.complete-section
DEBUG:botocore.hooks:Changing event name from before-parameter-buil

In [ ]:
bucket = s3.create_bucket(Bucket='bloc-kayak-z' , CreateBucketConfiguration={'LocationConstraint': 'eu-west-3'})

DEBUG:boto3.resources.action:Calling s3:create_bucket with {'Bucket': 'bloc-kayak-z', 'CreateBucketConfiguration': {'LocationConstraint': 'eu-west-3'}}
DEBUG:botocore.hooks:Event before-endpoint-resolution.s3: calling handler <function customize_endpoint_resolver_builtins at 0x7f69fc2b5f80>
DEBUG:botocore.hooks:Event before-endpoint-resolution.s3: calling handler <bound method S3RegionRedirectorv2.redirect_from_cache of <botocore.utils.S3RegionRedirectorv2 object at 0x7f69fc18a510>>
DEBUG:botocore.regions:Calling endpoint provider with parameters: {'Bucket': 'bloc-kayak-z', 'Region': 'us-east-1', 'UseFIPS': False, 'UseDualStack': False, 'ForcePathStyle': False, 'Accelerate': False, 'UseGlobalEndpoint': True, 'DisableAccessPoints': True, 'DisableMultiRegionAccessPoints': False, 'UseArnRegion': True}
DEBUG:botocore.regions:Endpoint provider result: https://bloc-kayak-z.s3.amazonaws.com
DEBUG:botocore.regions:Selecting from endpoint provider's list of auth schemes: "sigv4". User selected 

In [ ]:
# Uploading the csv file into the bucket

bucket.put_object(Key ='df_topp5' , Body= 'df_topp5.csv')


DEBUG:boto3.resources.action:Calling s3:put_object with {'Bucket': 'bloc-kayak-z', 'Key': 'df_topp5', 'Body': 'df_topp5.csv'}
DEBUG:botocore.hooks:Event before-endpoint-resolution.s3: calling handler <function customize_endpoint_resolver_builtins at 0x7f69fc2b5f80>
DEBUG:botocore.hooks:Event before-endpoint-resolution.s3: calling handler <bound method S3RegionRedirectorv2.redirect_from_cache of <botocore.utils.S3RegionRedirectorv2 object at 0x7f69fc18a510>>
DEBUG:botocore.regions:Calling endpoint provider with parameters: {'Bucket': 'bloc-kayak-z', 'Region': 'us-east-1', 'UseFIPS': False, 'UseDualStack': False, 'ForcePathStyle': False, 'Accelerate': False, 'UseGlobalEndpoint': True, 'DisableMultiRegionAccessPoints': False, 'UseArnRegion': True}
DEBUG:botocore.regions:Endpoint provider result: https://bloc-kayak-z.s3.amazonaws.com
DEBUG:botocore.regions:Selecting from endpoint provider's list of auth schemes: "sigv4". User selected auth scheme is: "None"
DEBUG:botocore.regions:Selected 

s3.Object(bucket_name='bloc-kayak-z', key='df_topp5')

#Data Warehouse PostgresSQL RDS


In [ ]:
from sqlalchemy import create_engine, text


In [ ]:
# Connecting to the sql database

DBHOST = 'kayak-zahra.cifns2dm8mpg.eu-west-3.rds.amazonaws.com'
DBUSER = 'postgres'
DBPASS = 'Zahra79126403'
DBNAME = ''

engine = create_engine(f"postgresql+psycopg2://{DBUSER}:{DBPASS}@{DBHOST}/{DBNAME}", echo=True)

In [ ]:
df_topp5.to_sql('df_kayak2', engine)

2022-11-29 12:39:11,681 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


INFO:sqlalchemy.engine.Engine:select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-11-29 12:39:11 [sqlalchemy.engine.Engine] INFO: select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


2022-11-29 12:39:11,696 INFO sqlalchemy.engine.Engine [cached since 88.94s ago] {'name': 'df_kayak2'}


INFO:sqlalchemy.engine.Engine:[cached since 88.94s ago] {'name': 'df_kayak2'}
2022-11-29 12:39:11 [sqlalchemy.engine.Engine] INFO: [cached since 88.94s ago] {'name': 'df_kayak2'}


2022-11-29 12:39:12,341 INFO sqlalchemy.engine.Engine BEGIN (implicit)


...

INFO:sqlalchemy.engine.Engine:INSERT INTO df_kayak2 (index, name, hotel, url, price, rating, "Temperature", "Temperature_max", "Temperature_min", "Humidity", lon_hotel, lat_hotel, address_hotel) VALUES (%(index)s, %(name)s, %(hotel)s, %(url)s, %(price)s, %(rating)s, %(Temperature)s, %(Temperature_max)s, %(Temperature_min)s, %(Humidity)s, %(lon_hotel)s, %(lat_hotel)s, %(address_hotel)s)
2022-11-29 12:39:13 [sqlalchemy.engine.Engine] INFO: INSERT INTO df_kayak2 (index, name, hotel, url, price, rating, "Temperature", "Temperature_max", "Temperature_min", "Humidity", lon_hotel, lat_hotel, address_hotel) VALUES (%(index)s, %(name)s, %(hotel)s, %(url)s, %(price)s, %(rating)s, %(Temperature)s, %(Temperature_max)s, %(Temperature_min)s, %(Humidity)s, %(lon_hotel)s, %(lat_hotel)s, %(address_hotel)s)


2022-11-29 12:39:13,228 INFO sqlalchemy.engine.Engine [generated in 0.00647s] ({'index': 0, 'name': 'Ariège', 'hotel': 'LE YESLOWBUS en fête', 'url': 'https://www.booking.com/hotel/fr/le-yeslowbus-en-fete.fr.html?label=gen173nr-1FCAEoggI46AdIDVgEaMkBiAEBmAENuAEYyAEP2AEB6AEB-AECiAIBqAIEuAKT9pecBsACAd ... (379 characters truncated) ... ocks=898265501_360475080_2_0_0&matching_block_id=898265501_360475080_2_0_0&sr_pri_blocks=898265501_360475080_2_0_0__9070&from=searchresults#hotelTmpl', 'price': '\xa091', 'rating': '10', 'Temperature': 6.0600000000000005, 'Temperature_max': 6.0600000000000005, 'Temperature_min': -2.075, 'Humidity': 58.0, 'lon_hotel': '1.37339700', 'lat_hotel': '43.15829880', 'address_hotel': 'Saubole, 09130 Carla-Bayle, France'}, {'index': 1, 'name': 'Ariège', 'hotel': 'LOFT24', 'url': 'https://www.booking.com/hotel/fr/loft24.fr.html?label=gen173nr-1FCAEoggI46AdIDVgEaMkBiAEBmAENuAEYyAEP2AEB6AEB-AECiAIBqAIEuAKT9pecBsACAdICJDdjOTRlNzE4 ... (397 characters truncated) ... atch

INFO:sqlalchemy.engine.Engine:[generated in 0.00647s] ({'index': 0, 'name': 'Ariège', 'hotel': 'LE YESLOWBUS en fête', 'url': 'https://www.booking.com/hotel/fr/le-yeslowbus-en-fete.fr.html?label=gen173nr-1FCAEoggI46AdIDVgEaMkBiAEBmAENuAEYyAEP2AEB6AEB-AECiAIBqAIEuAKT9pecBsACAd ... (379 characters truncated) ... ocks=898265501_360475080_2_0_0&matching_block_id=898265501_360475080_2_0_0&sr_pri_blocks=898265501_360475080_2_0_0__9070&from=searchresults#hotelTmpl', 'price': '\xa091', 'rating': '10', 'Temperature': 6.0600000000000005, 'Temperature_max': 6.0600000000000005, 'Temperature_min': -2.075, 'Humidity': 58.0, 'lon_hotel': '1.37339700', 'lat_hotel': '43.15829880', 'address_hotel': 'Saubole, 09130 Carla-Bayle, France'}, {'index': 1, 'name': 'Ariège', 'hotel': 'LOFT24', 'url': 'https://www.booking.com/hotel/fr/loft24.fr.html?label=gen173nr-1FCAEoggI46AdIDVgEaMkBiAEBmAENuAEYyAEP2AEB6AEB-AECiAIBqAIEuAKT9pecBsACAdICJDdjOTRlNzE4 ... (397 characters truncated) ... atching_block_id=844947701_3

2022-11-29 12:39:14,076 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT
2022-11-29 12:39:14 [sqlalchemy.engine.Engine] INFO: COMMIT


In [ ]:
# SQL Request to get the data from the RDS Data Warehouse
df = text("SELECT * FROM df_kayak2")
df_final = pd.read_sql(df, engine)


2022-11-29 12:39:25,848 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


INFO:sqlalchemy.engine.Engine:select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-11-29 12:39:25 [sqlalchemy.engine.Engine] INFO: select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


2022-11-29 12:39:25,856 INFO sqlalchemy.engine.Engine [cached since 103.1s ago] {'name': 'SELECT * FROM df_kayak2'}


INFO:sqlalchemy.engine.Engine:[cached since 103.1s ago] {'name': 'SELECT * FROM df_kayak2'}
2022-11-29 12:39:25 [sqlalchemy.engine.Engine] INFO: [cached since 103.1s ago] {'name': 'SELECT * FROM df_kayak2'}


2022-11-29 12:39:26,486 INFO sqlalchemy.engine.Engine SELECT * FROM df_kayak2


INFO:sqlalchemy.engine.Engine:SELECT * FROM df_kayak2
2022-11-29 12:39:26 [sqlalchemy.engine.Engine] INFO: SELECT * FROM df_kayak2


2022-11-29 12:39:26,491 INFO sqlalchemy.engine.Engine [generated in 0.00518s] {}


INFO:sqlalchemy.engine.Engine:[generated in 0.00518s] {}
2022-11-29 12:39:26 [sqlalchemy.engine.Engine] INFO: [generated in 0.00518s] {}


In [ ]:
df_final.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   index            82 non-null     int64  
 1   name             82 non-null     object 
 2   hotel            82 non-null     object 
 3   url              82 non-null     object 
 4   price            82 non-null     object 
 5   rating           82 non-null     object 
 6   Temperature      82 non-null     float64
 7   Temperature_max  82 non-null     float64
 8   Temperature_min  82 non-null     float64
 9   Humidity         82 non-null     float64
 10  lon_hotel        82 non-null     object 
 11  lat_hotel        82 non-null     object 
 12  address_hotel    82 non-null     object 
dtypes: float64(4), int64(1), object(8)
memory usage: 8.5+ KB


In [ ]:
df_final.price=df_final.price.apply(lambda x: x.replace("\xa01",''))


In [ ]:
df_final = df_final.astype({"lat_hotel": float,"lon_hotel": float,"price":float})

In [ ]:
fig = px.scatter_mapbox(df_final, lat="lat_hotel", lon="lon_hotel",
                  
                            mapbox_style="carto-positron",zoom=4, width = 800,  height =860,title = 'Location of top hotels in 5 different cities',color='Temperature',size='price'
                          
            )
fig.show(renderer="colab")